---
title: Jupyter Custom magic function
tags: 小书匠,jupyter,custom,extentions,magic,ipython
grammar_cjkRuby: true
renderNumberedHeading: true
---

[toc]

# Jupyter Custom magic function

## 简单示例

### 定义 line magic

In [1]:
from IPython.core.magic import (register_line_magic,
                                register_cell_magic)

In [2]:
@register_line_magic
def hello(line):
    if line == 'french':
        print("Salut tout le monde!")
    else:
        print("Hello world!")

In [3]:
%hello

Hello world!


In [4]:
%hello french

Salut tout le monde!


### Custom a cell magic command

In [5]:
import pandas as pd
from io import StringIO

@register_cell_magic
def mycsv(line, cell):
    # We create a string buffer containing the
    # contents of the cell.
    sio = StringIO(cell)
    # We use Pandas' read_csv function to parse
    # the CSV string.
    return pd.read_csv(sio)

In [6]:
%%mycsv
col1,col2,col3
0,1,2
3,4,5
7,8,9

,col1,col2,col3
0,0,1,2
1,3,4,5
2,7,8,9


### Saving extension

In [7]:
%%writefile csvmagic.py
import pandas as pd
from io import StringIO

def mycsv(line, cell):
    sio = StringIO(cell)
    return pd.read_csv(sio)

def load_ipython_extension(ipython):
    """This function is called when the extension is
    loaded. It accepts an IPython InteractiveShell
    instance. We can register the magic with the
    `register_magic_function` method of the shell
    instance."""
    ipython.register_magic_function(csv, 'cell')

Overwriting csvmagic.py


注意，csvmagic 是 extension 的名字，而 mycsv 是这个 extension 中包含的 magic command

In [8]:
%load_ext csvmagic

The csvmagic extension is already loaded. To reload it, use:
  %reload_ext csvmagic


In [9]:
%%mycsv
col1,col2,col3
0,1,2
3,4,5
7,8,9

,col1,col2,col3
0,0,1,2
1,3,4,5
2,7,8,9


## Config to autoload an extension

The Python extension module needs to be importable when using %load_ext. Here, our module is in the current directory. In other situations, it has to be in the Python path. It can also be stored in `~/.ipython/extensions`, which is automatically put in the Python path.

In [10]:
%%writefile ~/.ipython/extensions/csvmagic.py

import pandas as pd
from io import StringIO

def csv(line, cell):
    sio = StringIO(cell)
    return pd.read_csv(sio)

def load_ipython_extension(ipython):
    """This function is called when the extension is
    loaded. It accepts an IPython InteractiveShell
    instance. We can register the magic with the
    `register_magic_function` method of the shell
    instance."""
    ipython.register_magic_function(csv, 'cell')

Overwriting /Users/ed/.ipython/extensions/csvmagic.py


To ensure that our magic is automatically defined in our IPython profile, we can instruct IPython to load our extension automatically when a new interactive shell is launched. To do this, we have to open the `~/.ipython/profile_default/ipython_config.py` file and put 'csvmagic' in the `c.InteractiveShellApp.extensions` list. The csvmagic module needs to be importable. It is common to create a Python package that implements an IPython extension, which itself defines custom magic commands.

```
c.InteractiveShellApp.extensions = [ 'mycsv' ]
```

In [11]:
%%mycsv
col1,col2,col3
0,1,2
3,4,5
7,8,9

,col1,col2,col3
0,0,1,2
1,3,4,5
2,7,8,9


## 高级内容

### 类定义

可以看 References

# References
- 推荐。这个很清晰地说了如何定制 [IPython Cookbook - 1.4. Creating an IPython extension with custom magic commands](https://ipython-books.github.io/14-creating-an-ipython-extension-with-custom-magic-commands/)
- [Custom magic commands for Jupyter notebooks](https://gist.github.com/burchill/4adb9531a246e8c27752c1b19e0236bb)
- 官方文档，但是感觉不很清晰 [IPython extensions — IPython 7.22.0 documentation](https://ipython.readthedocs.io/en/stable/config/extensions/index.html)